<a href="https://colab.research.google.com/github/rubensdmp/GeoUy/blob/main/GetLoc%26LinksKronon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Script para extraer información geográfica de fotografías**

## **Resumen**

Este pequeño script extrae la información ubicación geográfica de fotografías y luego se crea un CSV con dicha ubicación y el link de google drive para acceder públicamente a las fotos.
Este CSV luego es cargado en R y a traves de la biblioteca MapView se plotean como popups las fotos en su ubicación original.

Desarrollado por [**Rubens Merlino**](https://www.linkedin.com/in/rubens-merlino-uy/)


In [97]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [121]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth) # Create GoogleDrive instance with authenticated GoogleAuth instance

folder_id = '18nDmE8ttKu9jC-zogZ1uyc5knRstxdtf'

str = "\'" + folder_id + "\'" + " in parents and trashed=false"


# Auto-iterate through all files in the root folder.
dir_list = drive.ListFile({'q': str}).GetList()


In [122]:
import os
import glob
import pandas as pd
import requests
!pip install exifread
import exifread

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [123]:
file_list=[]
for dir in dir_list:
    folder_id = dir['id']

    str = "\'" + folder_id + "\'" + " in parents and trashed=false"

    # Auto-iterate through all files in the root folder.
    internal_file = drive.ListFile({'q': str}).GetList()
    for file in internal_file:
        file['lastModifyingUserName'] = dir['title']

    file_list = file_list + internal_file



#file_list[0]

In [124]:
path = '/content/drive/MyDrive/07. Personal/11. Travesías & Campismo/FotosMapa/'
fms=[]
names=[]
lats=[]
lons=[]
caps=[]
rutas=[]

#for file1 in file_list:
#for file1 in glob.glob(path):
    

for file1 in file_list:
    fms.append(file1['lastModifyingUserName'])
    rutas.append('https://drive.google.com/uc?id=' + file1['id'])
    
    with open(path + file1['lastModifyingUserName'] + '/' + file1['title'], 'rb') as f:
    #with open('https://drive.google.com/uc?id=' + file1['id'], 'rb') as f:
        tags = exifread.process_file(f)
        lat_ref = tags["GPS GPSLatitudeRef"].printable
        lat = tags["GPS GPSLatitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
        lat = float(lat[0]) + float(lat[1]) / 60 + float(lat[2]) / float(lat[3]) / 3600
        
        if lat_ref != "N":
            lat = lat * (-1)
        lon_ref = tags["GPS GPSLongitudeRef"].printable
        lon = tags["GPS GPSLongitude"].printable[1:-1].replace(" ", "").replace("/", ",").split(",")
        lon = float(lon[0]) + float(lon[1]) / 60 + float(lon[2]) / float(lon[3]) / 3600
        
        if lon_ref != "E":
            lon = lon * (-1)
        
        names.append(f.name[60:])        
        lats.append(lat)
        lons.append(lon)
        caps.append(tags["EXIF DateTimeOriginal"])

  

In [125]:
#defino cabezales del CSV creado como un dataframe
#nombre foto / Ubicación
#from google.colab import files

df = pd.DataFrame()
df['formacion'] = fms
#df['name'] = names
df['lat'] = lats
df['lon'] = lons
df['captura'] = caps
df['ruta'] = rutas

#df = df.assign(Edad=None)

df.to_csv('/content/drive/My Drive/07. Personal/11. Travesías & Campismo/FotosMapa/Campismo.csv', encoding='utf-8-sig')

files.download("/content/drive/My Drive/07. Personal/11. Travesías & Campismo/FotosMapa/Campismo.csv")

df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,formacion,lat,lon,captura,ruta
0,2020 Rio Cebollati (R8-Averias),-33.781994,-54.660673,2020:01:02 18:17:09,https://drive.google.com/uc?id=1bl0qUmr7FUIyWD...
1,2020 Rio Cebollati (R8-Averias),-33.804264,-54.710971,2020:01:02 13:52:28,https://drive.google.com/uc?id=1PZi6jmYCkXeblM...
2,2020 Rio Cebollati (R8-Averias),-33.678830,-54.408206,2020:01:05 14:50:38,https://drive.google.com/uc?id=1FY7Tpi53GOfrvD...
3,2020 Rio Cebollati (R8-Averias),-33.678869,-54.408180,2020:01:05 14:50:34,https://drive.google.com/uc?id=1gUhf5dvdYZluG1...
4,2020 Rio Cebollati (R8-Averias),-33.763041,-54.493964,2020:01:04 14:16:21,https://drive.google.com/uc?id=1akQXMkvo14Vnt6...
